In [43]:
# -*- coding: utf-8 -*- 
import urllib , requests , sys , string   ,time , os
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from time import gmtime , strftime
#dlsite
#fkey作者來源key.txt
#fcomiclist結果key_dlsite.text

#輸出格式：
#dlsite
#!作者
#!總筆數
#==類別_數量_中文敘述
#類別_品名
#網址
'''
#木星在住DOJIN
<span class="icon_GAM" title="同人ゲーム">同人ゲーム</span>
<span class="icon_SOF" title="同人ソフト">同人ソフト</span>
<span class="icon_DOH" title="同人誌">同人誌</span>
GAM,SOF,DOH

<span class="icon_EVT" title="コミックマーケット86">コミックマーケット86</span>
C89

<span class="icon_ADL" title="18禁">18禁</span>

####
listtype=[[],[MDC,SCM,MNG],[],[IN2,INV,NR2,NRE,ET3,ET4,ACN,QIZ,ADV,RPG,TBL,DNV,SLN,TYP,STG,PZL,ETC],
[MOV],[SOU,MUS,ICG,IST,GSH],[GAM,SOF,DOH],[],[]]

2_MDC,SCM,MNG

4_IN2,INV,NR2,NRE,ET3,ET4,ACN,QIZ,ADV,RPG,TBL,DNV,SLN,TYP,STG,PZL,ETC
5_MOV

6_SOU,MUS,ICG,IST,GSH
7_GAM,SOF,DOH


#'''

#全轉半_含轉小寫
def Q2B(ustring):
    fs=u'０１２３４５６７８９ＱｑＷｗＥｅＲｒＴｔＹｙＵｕＩｉＯｏＰｐ\
ＡａＳｓＤｄＦｆＧｇＨｈＪｊＫｋＬｌＺｚＸｘＣｃＶｖＢｂＮｎＭｍ'
    hs=u'0123456789QqWwEeRrTtYyUuIiOoPpAaSsDdFfGgHhJjKkLlZzXxCcVvBbNnMm'
    rstr = ''
    for tm in ustring:
        if fs.find(tm)+1:
            tm = hs[fs.find(tm)]
        rstr = rstr + tm
    ustring = rstr
    ustring=ustring.lower()
    return ustring

#資料儲存
def save(fout , listdata , sdict , check=0):
    for tep in listdata:
        if sdict.get(tep):
            fout.write(sdict[tep]  + '\n')
    #return

def finddata(key, blink, check):
    #作者頁範例_http://www.dlsite.com/pro/work/=/product_id/VJ008602.html
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    rr=res.text
    rr=rr[rr.rfind('<div id=\"work_header\"'):rr.rfind('<!-- /work_story -->')]
    sou=BeautifulSoup(rr,"lxml")
    
    bdata=''
    #資訊欄位
    n1 = Q2B(sou.select('#work_right_inner')[0].text).encode('utf8').count(key)#作者出現次數
    if n1 > 0:
        tm=Q2B(sou.select('#work_right_inner')[0].text)#.replace('\r','')
        for temp in range(n1):
            t1=tm.rfind(u'\n',0,tm.find(key.decode('utf8')))
            t2=tm.find(u'\n',tm.find(key.decode('utf8')))
            bdata=bdata+'!'+tm[t1 : t2].strip(u' \n\r\t')
            tm=tm[t2:]
    #內容欄位
    n2 = Q2B(sou.select('.work_article.work_story')[0].text).encode('utf8').count(key)#作者出現次數
    if n2 > 0:
        tm=Q2B(sou.select('.work_article.work_story')[0].text).replace('\r','')
        for temp in range(n2):
            t1=tm.rfind(u'\n',0,tm.find(key.decode('utf8')))
            t2=tm.find(u'\n',tm.find(key.decode('utf8')))
            bdata=bdata+'!'+tm[t1 : t2].strip(u' \n\r\t')
            tm=tm[tm.find(key.decode('utf8'))+len(key.decode('utf8')):]
    nnum=n1+n2
    
    return nnum,bdata


#資料處理
def findbook(dictB, listdata, receive):
    link, key, p, pn, pnn, olddate=receive
    mlink = 'http://www.dlsite.com/books/'
    if len(link)<5:
        return dictB, listdata, pnn, olddate
    
    res = requests.get(link)
    res.encoding =  res.apparent_encoding
    r= res.text
    rt=u'<table class=\"work_1col_table\" cellspacing=\"0\">'
    r=r[r.find(rt)+len(rt):r.find(u'<script type=\'text/javascript\'>')]
    r=r[:r.rfind(u'</table>')]
    
    bn=r.count('work_1col_thumb')#頁作品數
    doc=r.split('<tr>\n<td colspan=\"3\">\n<hr class=\"work_border\" />\n</td>\n</tr>')
    #doc.pop()#多餘項
    #print bn,len(soup.select('tr')),len(soup.select('td'))
    
    print len(doc),bn
    
    listtype = [[], [], ['MDC','SCM','MNG'], [],
                ['IN2','INV','NR2','NRE','ACN','QIZ','ADV','RPG','TBL','DNV','SLN','TYP','STG','PZL','ETC'],
                ['MOV'], ['SOU','MUS','ICG','IST','GSH'], ['ET3','ET4','GAM','SOF','DOH'], [], []]
    #<span class="icon_ADL" title="18禁">18禁</span>
    
    a =0
    for tm in doc[:3]:
        if pnn==0:
            return dictB,listdata,oname,mindate,pnn,olddate
        pnn=pnn-1
        a=a+1
        print '\r',pnn,
        
        sou = BeautifulSoup(tm,"lxml")
        
        ctype = sou.find(class_='work_genre')
        cbook = sou.select('.work_name')[0].text.strip(' \n')
        cdate = sou.select('.sales_date')[0].text.strip(' \n')
        cname = sou.select('.maker_name')[0].text.strip(' \n')
        check = 0
        
        blink = ''
        blink = sou.select('.work_name')[0].select('a')[0].get('href')
        
        
        #&nbsp;_\xa0
        #print repr(cname)
        
        #類型處理
        for t in listtype:
            for tt in ctype.select('span'):
                if tt.get('class')[0][5:] in t:
                    check=listtype.index(t)
                    print 'check',check
        if check==0:
            continue
        
        #作者處理
        if u'\xa0/\xa0' in cname:#出版商
            cname=cname[:cname.rfind(u'\xa0/\xa0')]
        nname=cname.split(u'\xa0')
        if (len(nname) > 1) and (check==2):#多人漫
            check=3
        
        bname=''
        c=0
        for tt in nname:
            if key.decode('utf8') == tt:
                c=1
                #print 'find'
            #elif (key.decode('utf8') in tt) and (c==0):
                #c=2
                #print 'find#'
            bname=bname+tt+'_'
        bname=bname.strip('_')
        if c==0:# and (check==2 or check==3 or check==7)
            check=8
            #print 'X'
            nnum,bdata = finddata(key, blink, check)
            print nnum,bdata
            #=finddata(blink)
        
        
        '''
        #予告開始日 : 	2015年04月24日
        #販売日: 2015年08月21日
        if u'予告開始日' in cdate:
            cdate=cdate[7:].replace(u'年','').replace(u'月','').replace(u'日','')
            check=1
        else:
            cdate=cdate[5:].replace(u'年','').replace(u'月','').replace(u'日','')'''
        
        #日期處理
        cdate=cdate[5:].replace(u'年','').replace(u'月','').replace(u'日','')
        if len(cdate) < 4:#無日期
            cdate = '00000000'#填入日期
        date = cdate
        while listdata.count(date):#重複日期判斷
            date = date[:-2] + str(int(date[-2:]) + 1).rjust(2,'0')#日期+1_十位數填0
        listdata.append(date)
        
        #print ctype,'###'#cbook,cdate
        
        print check,bname,date
        print blink
        continue
        
        
        #作品處理
        #日期_品名
        #類型_nnum1_bdate1_nnum2_bdate2
        cfind=bfind(blink)
        #print '########catch:',len(cfind)#,cfind[:]
        ctype=cfind[0]
        cnum1=str(cfind[1]).decode('utf8')
        cdate1=cfind[2]
        cnum2=str(cfind[3]).decode('utf8')
        cdate2=cfind[4]
        check=cfind[5]
        
        
        #寫入dict
        #參考1參考2，無筆數則不紀錄。
        if check == 1:#新
            #日期_類型_品名_
            #!參考1!連結
            book=data+'_'+dtype+'_'+book+'_\n!'+cnum1+'!'+blink
            dict1.setdefault(data,book)
        elif check == 2:#漫單
            #類型_品名_作者資料1_
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict2.setdefault(data,book)
        elif check == 3:#漫多
            #類型_品名_作者資料1_
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict3.setdefault(data,book)
        elif check == 4:#漫同人
            #類型_品名_作者資料1_
            book=dtype+'_'+book+'_'+cdate1+'_'
            dict4.setdefault(data,book)
        elif check == 5:#它
            #類型_品名_作者資料1_
            #!參考1!參考2!連結
            book=dtype+'_'+book+'_'+cdate1+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict5.setdefault(data,book)
        elif check == 6:#漫
            #類型_品名_
            #作者資料2_
            #!參考1!參考2!連結
            book=dtype+'_'+book+'_\n'+cdate2+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict6.setdefault(data,book)
        elif check==7:#它
            #類型_品名_
            #作者資料2_
            #!參考1!參考2!連結
            book=dtype+'_'+book+'_\n'+cdate2+'_\n!'+cnum1+'!'+cnum2+'!'+blink
            dict7.setdefault(data,book)
        #print check,'_',book
        #'''
        
        if a==(bn):
            break
        #continue
    print '.'
    #print '========'
    return dictB, listdata, pnn, olddate

########
def main(key='',ucheck=0,pn=0,nlink=''):
    #ucheck_0建檔_1更新_2直讀
    
    pnum = 30#頁顯示數量
    mlink = 'http://www.dlsite.com/books/'
    
    olddate=''
    if ucheck != 2:
        #key=作者
        key=key.lower()
        key2 = urllib.quote(key.decode('utf8').encode('shift_jis'))#當sjis輸出utf8的url
        key3 = urllib.unquote(key2.decode('shift_jis').encode('utf8'))#utf8的url翻sjis
        #網址用
        key4 = urllib.quote(key)

        #檢查BOM
        if '%EF%BB%BF' in urllib.quote(key):
            print 'BOM！'

        link = "http://www.dlsite.com/maniax/fsr/=/language/jp/\
sex_category%5B0%5D/male/keyword/"+key4+"/ana_flg/all/\
order%5B0%5D/release_d/genre_and_or/or/options_and_or/or/per_page/30/show_type/n/page/1"
        
        res = requests.get(link)
        res.encoding =  res.apparent_encoding#亂碼處理
        r= res.text
        r=r[r.find(u'<!-- ▼mainContents -->'):r.find(u'<!-- ▲mainContents -->')+len(u'<!-- ▲mainContents -->')]
        #soup = BeautifulSoup(r ,"lxml")#,  parse_only=only_a_tags)
        soup = BeautifulSoup(res.text,"html.parser")
        
        pn=0
        if soup.find(class_='page_total'):
            pn = int(soup.select('.page_total')[0].strong.text)#資料筆數
        
        #updata
        if (ucheck == 1) and (os.path.isfile('output/' + key.decode('utf8') + '_dlsitev1.txt')):#更新與欲輸入檔案存在
            fupdata = open('output/'+key.decode('utf8') + '_dlsitev1.txt', 'r+')
            rf=list(fupdata)
            kc=rf[1]
            if kc[1:len(key)+1] == key:
                oldn=int(rf[2][10:rf[2].find('_')])#建檔時筆數
                olddate=rf[2][rf[2].find('_')+1:rf[2].find('_')+11]
                olddate=olddate+str(9999)

                #更新筆數疊加
                for temp in range(rf.count('%\n')-1):#多一項
                    ub=rf.index('%\n')+2#位移
                    rf[rf.index('%\n')]=''#首項處理
                    oldnn=int(rf[ub][7:rf[ub].find('_')])
                    oldn=oldn+oldnn
                    olddate=rf[ub][rf[ub].find('_')+1:rf[ub].find('_')+11]
                    
                olddate=olddate.replace('/','')
                pn=pn-oldn
                print '更新筆數'+str(pn)

            if pn==0:#需要更新數為0跳出
                fupdata.seek(0, 2)
                fupdata.write('!updata\n!筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d")+'\n%\n')
                fupdata.close()
                return 

    #資料筆數_是否數字
    if pn > 0:
        pnn=pn
        if int(pn) > pnum:
            print 'BIG'

        if ucheck == 1:
            fupdata.seek(0, 2)
            fupdata.write('!updata\n!筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d,%H:%M")+'->')
        else:
            fout = open('output/'+key.decode('utf8') + '_dlsitev1.txt', 'w')#寫入模式開檔
            fout.write('dlsite\n')
            fout.write('!' + key + '_!'+link+'\n')
            fout.write('!總筆數' + str(pn).encode('utf8') +'_'+ strftime("%Y/%m/%d,%H:%M")+'->')
        
        print '==dlsite' , pn, '筆'
        time.sleep(1)
        p = 0#頁
        #建空輸出用字典與陣列
        dict1={}#新
        dict2={}#漫單_マンガ
        dict3={}#漫多
        dict4={}#イラスト＋ノベル_ノベル_ゲーム
        dict5={}#動画
        dict6={}#其他_音声_音楽_イラスト集_画集
        dict7={}#同人_その他
        #dict8={}#一般向
        dict8={}#它_內容欄
        dictB={
            'dict1':dict1,'dict2':dict2,'dict3':dict3,'dict4':dict4,
            'dict5':dict5,'dict6':dict6,'dict7':dict7,'dict8':dict8
            #'dict9':dict9
        }
        listdata = []
        #check=0

        #資料處理
        while (int(pn) - p * pnum) > 0:
            p = p + 1
            print 'page:' + str(p)
            
            nlink = link[:link.rfind('/')+1]+str(p)
            send=[nlink, key, p, pn, pnn, olddate]
            dictB, listdata, pnn, olddate = findbook(dictB, listdata, send)#資料處理
            time.sleep(1)
        sys.exit()################

        #日期排序
        #listdata.sort()
        
        listw=['==new_' + str(len(dictB["dict1"])) +'_新\n',
               '==book_' + str(len(dictB["dict2"])) +'_漫單\n',
               '==magazine_' + str(len(dictB["dict3"])) +'_漫多\n',
               '==djs_' + str(len(dictB["dict4"])) +'_同人\n',
               '==other_' + str(len(dictB["dict5"])) +'_它\n',
               '==obook_' + str(len(dictB["dict6"])) +'_漫\n',
               '==oother_' + str(len(dictB["dict7"])) +'_它\n'
              ]
        
        #紀錄筆數
        nb=0
        for w in range(len(listw)):
            nb= nb+ len(dictB["dict"+str(w+1)])
        
        #輸出
        if ucheck == 1:
            fupdata.write(strftime("%H:%M")+'_'+str(nb)+'\n')
            if len(olddate[8:]) > 0:#日期與資料不吻合，可能含舊資料
                fupdata.write('!舊資料'+olddate[8:]+'\n')
            for w in range(len(listw)):
                if len(dictB["dict"+str(w+1)]) > 0:
                    fupdata.write(listw[w])
                    save(fupdata , listdata , dictB["dict"+str(w+1)],w+1)
            fupdata.write('%\n')
            fupdata.close()
            
        else:
            fout.write(strftime("%H:%M")+'_'+str(nb)+'\n')
            for w in range(len(listw)):
                fout.write(listw[w])
                save(fout , listdata , dictB["dict"+str(w+1)],w+1)
            fout.write('%\n')
            fout.close()
        
        #sys.exit()################
        print 'ok'
    elif pn:
        print 'No Date'
    #
    return
####
if __name__ == '__main__':
    #main()
    main(key='木星在住',ucheck=0)
    #main(key='hisasi',ucheck=0)

    #已知作者頁面，給定參數建檔。(跳過特殊符號用)
    #main(key='木星在住',ucheck=2,pn=78,nlink='http://www.doujinshi.org/browse/author/36341/Mokusei-Zaijuu/')
    

BIG
==dlsite 44 筆
page:1
30 30
43 check 2
2 木星在住 20141212
http://www.dlsite.com/books/work/=/product_id/BJ048285.html
42 check 2
3 あり_よしじまあたる_2号_木星在住_あさぎ龍_RED-RUM 20141213
http://www.dlsite.com/books/work/=/product_id/BJ048283.html
41 check 2
check 7
1 !木星在住
8 株式会社虎の穴 20141102
http://www.dlsite.com/maniax/work/=/product_id/RJ133093.html
.
page:2
14 14
40 check 2
check 7
1 !木星在住dojin
8 木星在住DOJIN 20130128
http://www.dlsite.com/maniax/work/=/product_id/RJ109912.html
39 check 2
check 7
1 !木星在住dojin
8 木星在住DOJIN 20130127
http://www.dlsite.com/maniax/work/=/product_id/RJ109910.html
38 check 2
check 7
1 !木星在住dojin
8 木星在住DOJIN 20121203
http://www.dlsite.com/maniax/work/=/product_id/RJ106733.html
.


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [37]:
import urllib , requests , sys ,string ,time
from bs4 import BeautifulSoup
from bs4 import SoupStrainer

def finddata(key, blink, check):
    #作者頁範例_http://www.dlsite.com/pro/work/=/product_id/VJ008602.html
    res = requests.get(blink)
    res.encoding =  res.apparent_encoding
    rr=res.text
    rr=rr[rr.rfind('<div id=\"work_header\"'):rr.rfind('<!-- /work_story -->')]
    sou=BeautifulSoup(rr,"lxml")
    
    bdata=''
    #資訊欄位
    n1 = Q2B(sou.select('#work_right_inner')[0].text).encode('utf8').count(key)#作者出現次數
    if n1 > 0:
        tm=Q2B(sou.select('#work_right_inner')[0].text)#.replace('\r','')
        for temp in range(n1):
            #print '#',temp
            t1=tm.rfind(u'\n',0,tm.find(key.decode('utf8')))
            t2=tm.find(u'\n',tm.find(key.decode('utf8')))
            bdata=bdata+'!'+tm[t1 : t2].strip(u' \n\r\t')#+u'\n'
            tm=tm[t2:]
    #內容欄位
    n2 = Q2B(sou.select('.work_article.work_story')[0].text).encode('utf8').count(key)#作者出現次數
    if n2 > 0:
        tm=Q2B(sou.select('.work_article.work_story')[0].text).replace('\r','')
        for temp in range(n2):
            #print '#',temp
            t1=tm.rfind(u'\n',0,tm.find(key.decode('utf8')))
            t2=tm.find(u'\n',tm.find(key.decode('utf8')))
            #print t1,t2
            bdata=bdata+'!'+tm[t1 : t2].strip(u' \n\r\t')#+u'\n'
            tm=tm[tm.find(key.decode('utf8'))+len(key.decode('utf8')):]
    nnum=n1+n2
    print len(res.text), len(str(sou)), 'nnum', nnum
    print bdata
    
    #print sou
    #return
blink='http://www.dlsite.com/maniax/work/=/product_id/RJ133093.html'
blink='http://www.dlsite.com/pro/work/=/product_id/VJ008602.html'
key='木星在住'
key='hisasi'
check=2
finddata(key,blink,check)

62682 11297 nnum 1
!その他 : 原作：『小悪魔カノジョ』（hisasi）
